# 1：多分类和神经网络
### **by:MLZZY**
**实验描述：实现手写数字（0到9）的识别。扩展之前的逻辑回归，并将其应用于一对多的分类。
数据集是一个MATLAB格式的.m文件，其中包含5000个20*20像素的手写字体图像，以及对应的数字（数字0的y值为10）**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from scipy.io import loadmat
#用于生成评价报告
from sklearn.metrics import classification_report

In [3]:
data=loadmat('/home/mw/input/andrew_ml_ex33507/ex3data1.mat')
data,data['X'].shape,data['y'].shape

({'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
  '__version__': '1.0',
  '__globals__': [],
  'X': array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]),
  'y': array([[10],
         [10],
         [10],
         ...,
         [ 9],
         [ 9],
         [ 9]], dtype=uint8)},
 (5000, 400),
 (5000, 1))

In [4]:
#数据的可视化（随机展示100个数据）
sample_idx=np.random.choice(np.arange(data['X'].shape[0]),100)
sample_images=data['X'][sample_idx,:]
sample_images

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [5]:
fig,ax_array=plt.subplots(nrows=10,ncols=10,sharey=True,sharex=True,figsize=(12,12))
for r in range(10):
    for c in range(10):
        ax_array[r,c].matshow(np.array(sample_images[10*r+c].reshape((20,20))).T,cmap=matplotlib.cm.binary)
#不显示x轴和y轴
plt.xticks(np.array([]))
plt.yticks(np.array([]))

([], <a list of 0 Text yticklabel objects>)

<Figure size 864x864 with 100 Axes>

In [6]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [7]:
def cost(theta,X,y,lamda):
    theta=np.matrix(theta)
    X=np.matrix(X)
    y=np.matrix(y)
    first=np.multiply(-y,np.log(sigmoid(X*theta.T)))
    second=np.multiply((1-y),np.log(1-sigmoid(X*theta.T)))
    reg=(lamda/(2*len(X)))*np.sum(np.power(theta[:,1:theta.shape[1]],2))
    return np.sum(first-second)/len(X)+reg

In [8]:
def gradient(theta,X,y,lamda):
    theta=np.matrix(theta)
    X=np.matrix(X)
    y=np.matrix(y)
    parameters=int(theta.ravel().shape[1])
    error=sigmoid(X*theta.T)-y
    grad=((X.T*error)/len(X)).T+((lamda/len(X))*theta)
    grad[0,0]=np.sum(np.multiply(error,X[:,0]))/len(X)
    return np.array(grad).ravel()

In [9]:
from scipy.optimize import minimize
def one_vs_all(X,y,num_labels,lamda):
    rows=X.shape[0]
    params=X.shape[1]
    all_theta=np.zeros((num_labels,params+1))
    #进行插入全1列的常规操作
    X=np.insert(X,0,values=np.ones(rows),axis=1)
    #i表示数字1——10(注意：该数据集中数字0的label为10)
    for i in range(1,num_labels+1):
        theta=np.zeros(params+1)
        y_i=np.array([1 if label==i else 0 for label in y])
        y_i=np.reshape(y_i,(rows,1))
        #最小化目标函数
        #fun：传入的是cost函数，就是自己写的计算代价的函数
        #x0：传入的是theta的初始值，初始化后的theta
        #method：所要使用的优化方法，如TNC、BFGS等
        #jac：传入的是gradient方法，即计算梯度的方法
        #args：梯度计算函数的参数（除了theta之外的按顺序写）
        #options：有两个选项，{‘maxiter’:100}可以控制迭代次数；{‘disp’:True}可以打印一些运行细节
        fmin=minimize(fun=cost,x0=theta,args=(X,y_i,lamda),method='TNC',jac=gradient)
        all_theta[i-1,:]=fmin.x
    return all_theta

In [10]:
rows=data['X'].shape[0]
params=data['X'].shape[1]
all_theta=np.zeros((10,params+1))
X=np.insert(data['X'],0,values=np.ones(rows),axis=1)
theta=np.zeros(params+1)
y_0=np.array([1 if lable==0 else 0 for lable in data['y']])
y_0=np.reshape(y_0,(rows,1))
#注：theta是一维数组，因此当它被转换为计算梯度的代码中的矩阵时，它变为（1×401）矩阵。
X.shape, y_0.shape, theta.shape, all_theta.shape

((5000, 401), (5000, 1), (401,), (10, 401))

In [11]:
np.unique(data['y'])#其中10代表数字0

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

In [29]:
all_theta=one_vs_all(data['X'],data['y'],10,1)
all_theta

array([[-2.38373823e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.30440684e-03, -7.49607957e-10,  0.00000000e+00],
       [-3.18277385e+00,  0.00000000e+00,  0.00000000e+00, ...,
         4.46416745e-03, -5.08967467e-04,  0.00000000e+00],
       [-4.79656036e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.87471064e-05, -2.47976297e-07,  0.00000000e+00],
       ...,
       [-7.98398219e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -8.95642491e-05,  7.22603652e-06,  0.00000000e+00],
       [-4.57124969e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.33504169e-03,  9.98035730e-05,  0.00000000e+00],
       [-5.40535662e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.16457336e-04,  7.86968213e-06,  0.00000000e+00]])

In [12]:
#使用训练完毕的分类器预测每个图像的标签。 
#计算每个类的类概率，对于每个训练样本，输出类标签为具有最高概率的类。
def predict_all(X,all_theta):
    rows=X.shape[0]
    params=X.shape[1]
    num_lables=all_theta.shape[0]
    X=np.insert(X,0,values=np.ones(rows),axis=1)
    X=np.matrix(X)
    all_theta=np.matrix(all_theta)
    h=sigmoid(X*all_theta.T)
    #找到每一行的最大值的下标
    h_argmax=np.argmax(h,axis=1)
    #因为h数组是零索引的，所以上述找到的最大值的下标是相对于0而言的，所以需要进行如下的加1操作
    h_argmax=h_argmax+1
    return h_argmax

In [13]:
y_pred=predict_all(data['X'],all_theta)
print(classification_report(data['y'],y_pred))

              precision    recall  f1-score   support

           1       0.10      1.00      0.18       500
           2       0.00      0.00      0.00       500
           3       0.00      0.00      0.00       500
           4       0.00      0.00      0.00       500
           5       0.00      0.00      0.00       500
           6       0.00      0.00      0.00       500
           7       0.00      0.00      0.00       500
           8       0.00      0.00      0.00       500
           9       0.00      0.00      0.00       500
          10       0.00      0.00      0.00       500

    accuracy                           0.10      5000
   macro avg       0.01      0.10      0.02      5000
weighted avg       0.01      0.10      0.02      5000



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


**神经网络**
**输入是图片的像素值，20*20像素的图片有400个输入层单元，不包括需要额外添加的常数项。
数据集提供了训练好的神经网络的参数theta1，theta2，有25个隐层单元和10个输出单元**

In [14]:
weight = loadmat("/home/kesci/input/andrew_ml_ex33507/ex3weights.mat")
theta1, theta2 = weight['Theta1'], weight['Theta2']
theta1.shape, theta2.shape

((25, 401), (10, 26))

In [15]:
# 插入常数项
X2 = np.matrix(np.insert(data['X'], 0, values=np.ones(X.shape[0]), axis=1))
y2 = np.matrix(data['y'])
X2.shape, y2.shape

((5000, 401), (5000, 1))

In [16]:
a1=X2
z2=a1*theta1.T
z2.shape

(5000, 25)

In [17]:
a2=sigmoid(z2)
a2.shape

(5000, 25)

In [18]:
a2=np.insert(a2,0,values=np.ones(a2.shape[0]),axis=1)
z3=a2*theta2.T
z3.shape

(5000, 10)

In [19]:
a3=sigmoid(z3)
a3

matrix([[1.12661530e-04, 1.74127856e-03, 2.52696959e-03, ...,
         4.01468105e-04, 6.48072305e-03, 9.95734012e-01],
        [4.79026796e-04, 2.41495958e-03, 3.44755685e-03, ...,
         2.39107046e-03, 1.97025086e-03, 9.95696931e-01],
        [8.85702310e-05, 3.24266731e-03, 2.55419797e-02, ...,
         6.22892325e-02, 5.49803551e-03, 9.28008397e-01],
        ...,
        [5.17641791e-02, 3.81715020e-03, 2.96297510e-02, ...,
         2.15667361e-03, 6.49826950e-01, 2.42384687e-05],
        [8.30631310e-04, 6.22003774e-04, 3.14518512e-04, ...,
         1.19366192e-02, 9.71410499e-01, 2.06173648e-04],
        [4.81465717e-05, 4.58821829e-04, 2.15146201e-05, ...,
         5.73434571e-03, 6.96288990e-01, 8.18576980e-02]])

In [20]:
y_pred2=np.argmax(a3,axis=1)+1
y_pred2.shape

(5000, 1)

In [21]:
print(classification_report(y2,y_pred2))

              precision    recall  f1-score   support

           1       0.97      0.98      0.98       500
           2       0.98      0.97      0.98       500
           3       0.98      0.96      0.97       500
           4       0.97      0.97      0.97       500
           5       0.97      0.98      0.98       500
           6       0.98      0.99      0.98       500
           7       0.98      0.97      0.97       500
           8       0.98      0.98      0.98       500
           9       0.97      0.96      0.96       500
          10       0.98      0.99      0.99       500

    accuracy                           0.98      5000
   macro avg       0.98      0.98      0.98      5000
weighted avg       0.98      0.98      0.98      5000

